In [1]:
import secrets
import json
from pathlib import Path
import pprint
import pdb
from typing import Any
import random 


import os
print(os.getcwd())
import sys
sys.path.append('../')

from etl import markdown, pdfs, shared, videos

import docstore
import vecstore
from utils import pretty_log

pp = pprint.PrettyPrinter(indent=2)

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline, TextStreamer
import json
import textwrap
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
import langchain
import time

from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

%load_ext autoreload
%autoreload 2
from QuestionAndAnswerUtils import *

In [3]:
securityChatbot = QuestionAndAnsweringCustomLlama3(QuestionRewritingPrompt=QuestionAndAnsweringCustomLlama2.QUESTION_REWRITING_TYPE.QUESTION_REWRITING_DEFAULT,
                                 QuestionAnsweringPrompt=QuestionAndAnsweringCustomLlama3.SECURITY_PROMPT_TYPE.PROMPT_TYPE_SECURITY_OFFICER_WITH_RAG_MEMORY_NOSOURCES,
                                 ModelType=QuestionAndAnsweringCustomLlama3.LLAMA3_VERSION_TYPE.LLAMA3_13B,
                                debug=True, streamingOnAnotherThread=False, demoMode=True, noInitialize=True)

securityChatbot.initializePromptTemplates()
securityChatbot.initializeLLMTokenizerandEmbedder()


In [8]:
pipe = pipeline("text-generation",
                                               model=securityChatbot.model,
                                               tokenizer=securityChatbot.tokenizer,
                                               torch_dtype=torch.float16,
                                               device_map="auto",
                                               max_new_tokens=512,
                                               do_sample=True,
                                               temperature=0.9,
                                                top_p=0.95,
                                               min_length=None,
                                               num_return_sequences=1,
                                               repetition_penalty=1.0,
                                               # The parameter for repetition penalty. 1.0 means no penalty.
                                               #length_penalty=1,
                                               # [optional] Exponential penalty to the length that is used with beam-based generation.
                                               eos_token_id=securityChatbot.tokenizer.eos_token_id,
                                               pad_token_id=securityChatbot.tokenizer.eos_token_id,
                                               streamer=securityChatbot.streamer,
                                               )

# Create the llm here
llm_custom = HuggingFacePipeline(pipeline=pipe)

def create_prompt_with_rag():
    qwithrag_template = get_prompt("Given the following information: {retrieved_info}, respond to: {question}")
    qwithrag_prompt = PromptTemplate(template=qwithrag_template,
                                    input_variables=["retrieved_info", "question"])
    return qwithrag_prompt
    
responseCheckPrompt = create_prompt_with_rag()

responseCheckChain = LLMChain(prompt=responseCheckPrompt, llm=llm_custom)


test_question_with_rag(chain_instance=responseCheckChain)

    import re
from statistics import mode

def gen_answer():
    response = completion(
        "John found that the average of 15 numbers is 40."
        "If 10 is added to each number then the mean of the numbers is?"
        "Report the answer surrounded by three backticks, for example: ```123```",
        model = DEFAULT_MODEL, #LLAMA2_70B
    )
    match = re.search(r'```(\d+)```', response)
    if match is None:
        return None
    return match.group(1)

answers = [gen_answer() for i in range(5)]

print(
    f"Answers: {answers}\n",
    f"Final answer: {mode(answers)}",
    )

complete_and_print(
    """
    # Python code to calculate: ((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
    """,
    model="meta/codellama-34b:67942fd0f55b66da802218a19a8f0e1d73095473674061a6ea19f2dc8c053152"
)

complete_and_print("""
Calculate the answer to the following math problem:

((-5 + 93 * 4 - 0) * (4^4 + -7 + 0 * 5))
""")

MENLO_PARK_TEMPS = {
    "2023-12-11": "52 degrees Fahrenheit",
    "2023-12-12": "51 degrees Fahrenheit",
    "2023-12-13": "51 degrees Fahrenheit",
}


def prompt_with_rag(retrived_info, question):
    complete_and_print(
        f"Given the following information: '{retrived_info}', respond to: '{question}'"
    )


def ask_for_temperature(day):
    temp_on_day = MENLO_PARK_TEMPS.get(day) or "unknown temperature"
    prompt_with_rag(
        f"The temperature in Menlo Park was {temp_on_day} on {day}'",  # Retrieved fact
        f"What is the temperature in Menlo Park on {day}?",  # User question
    )


ask_for_temperature("2023-12-12")
# "Sure! The temperature in Menlo Park on 2023-12-12 was 51 degrees Fahrenheit."

ask_for_temperature("2023-07-18")


### Limiting Extraneous Tokens

A common struggle is getting output without extraneous tokens (ex. "Sure! Here's more information on...").

Check out this improvement that combines a role, rules and restrictions, explicit instructions, and an example:


complete_and_print(
    "Give me the zip code for Menlo Park in JSON format with the field 'zip_code'",
    model = LLAMA3_70B_CHAT,
)
# Likely returns the JSON and also "Sure! Here's the JSON..."

complete_and_print(
    """
    You are a robot that only outputs JSON.
    You reply in JSON format with the field 'zip_code'.
    Example question: What is the zip code of the Empire State Building? Example answer: {'zip_code': 10118}
    Now here is my question: What is the zip code of Menlo Park?
    """,
    model = LLAMA3_70B_CHAT,
)
# "{'zip_code': 94025}"


In [ ]:
securityChatbot = QuestionAndAnsweringCustomLlama3(QuestionRewritingPrompt=QuestionAndAnsweringCustomLlama2.QUESTION_REWRITING_TYPE.QUESTION_REWRITING_DEFAULT,
                                 QuestionAnsweringPrompt=QuestionAndAnsweringCustomLlama3.SECURITY_PROMPT_TYPE.PROMPT_TYPE_SECURITY_OFFICER_WITH_RAG_MEMORY_NOSOURCES,
                                 ModelType=QuestionAndAnsweringCustomLlama2.LLAMA2_VERSION_TYPE.LLAMA2_7B_chat,
                                debug=True, streamingOnAnotherThread=True, demoMode=True, noInitialize=False)



securityChatbot.test_vectorDatasets_similarityScores_and_responses_no_memory(run_llm_chain=False)

securityChatbot.ask_question_and_streamtoconsole("What models use human instructions?")
securityChatbot.ask_question_and_streamtoconsole("Which are the advantage of each of these models?")
securityChatbot.ask_question_and_streamtoconsole("What are the downsides of your last model suggested above ?")



In [ ]:

securityChatbot.simulate_raq_question("can you show me a resource utilization graph comparison between a normal session and current situation?", run_llm_chain=True)


In [ ]:
securityChatbot.simulate_raq_question("What Smart IOT applications are developed by Rares, Alin, and Ciprian Paduraru?", run_llm_chain=True)

In [ ]:
start=time.time()
securityChatbot.ask_question("Give me some indications to solve a denial of service attack.")
securityChatbot.ask_question("Give me a few tools for this please")
securityChatbot.ask_question("Which one works best on Windows machines?")


end=time.time()

print(f"Total time: {end-start}")